First let's import pandas and numpy libraries to handle spreadsheets and dataframes, matplotlib to display basic plots and Prophet for predictions.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import fbprophet as Prophet


Read in the main spreadsheet that we are going to use. This contains ~20years worth of real estate data for many US states. We only be using a small sample of this data for this tuorial.

In [ ]:
state_time_series = pd.read_csv(r'dataset/State_time_series.csv')
# Make sure the date is interpreted correctly.
state_time_series.Date = pd.to_datetime(state_time_series.Date)
# Add another column just for year 
state_time_series['year']= state_time_series.Date.dt.year

# Display the final few rows
# to get an idea about how our dataset looks like
state_time_series.tail()


Let's sure that we are operating on valid data by getting all the states whose Zillow housing value index and median sold price columns are not null.

Then prepare a new spreadsheet with just these valid states in them.

In [ ]:
states = set(state_time_series[~state_time_series['ZHVI_AllHomes'].isnull() & ~state_time_series['MedianSoldPrice_AllHomes'].isnull()]['RegionName'].values)

state_time_series_valid = state_time_series[state_time_series['RegionName'].isin(states)].copy()

state_time_series_valid.tail()

Get top and bottom 5 costly states

In [ ]:
costliest_states = state_time_series_valid[['RegionName', 'ZHVI_AllHomes']].groupby('RegionName').max().sort_values(by=['ZHVI_AllHomes'], ascending=False)[:5].index.values.tolist()
print(costliest_states)
cheapest_states = state_time_series_valid[['RegionName', 'ZHVI_AllHomes']].groupby('RegionName').max().sort_values(by=['ZHVI_AllHomes'], ascending=True)[:5].index.values.tolist()
print(cheapest_states)

In [ ]:
costliest_time_series = state_time_series_valid[state_time_series_valid.RegionName.isin(costliest_states)]
costliest_time_series.tail()


In [ ]:
cheapest_time_series = state_time_series_valid[state_time_series_valid.RegionName.isin(cheapest_states)]
cheapest_time_series.tail()


In [ ]:
costliest_mean_sale_price = costliest_time_series.groupby([costliest_time_series.year, costliest_time_series.RegionName])['ZHVI_AllHomes'].mean().dropna().reset_index(name='MedianSoldPrice_AllHomes')
costliest_mean_sale_price


In [ ]:
cheapest_mean_sale_price = cheapest_time_series.groupby([cheapest_time_series.year, cheapest_time_series.RegionName])['ZHVI_AllHomes'].mean().dropna().reset_index(name='MedianSoldPrice_AllHomes')
cheapest_mean_sale_price


In [ ]:
costliest_mean_prices_pivot = costliest_mean_sale_price.pivot(index='year', columns='RegionName', values='MedianSoldPrice_AllHomes')

costliest_mean_prices_pivot


In [ ]:
fte_graph = costliest_mean_prices_pivot.plot(figsize=(20,10))
plt.pyplot.gca().xaxis.set_major_locator(plt.ticker.MaxNLocator(integer=True))
plt.pyplot.ylabel('Average SoldPrice')
plt.pyplot.xlabel('Year')


In [ ]:
cheapest_mean_prices_pivot = cheapest_mean_sale_price.pivot(index='year', columns='RegionName', values='MedianSoldPrice_AllHomes')

cheapest_mean_prices_pivot


In [ ]:
fte_graph = cheapest_mean_prices_pivot.plot(figsize=(20,10))
plt.pyplot.gca().xaxis.set_major_locator(plt.ticker.MaxNLocator(integer=True))
plt.pyplot.ylabel('Average SoldPrice')
plt.pyplot.xlabel('Year')


In [ ]:
#Let's prepare a dataframe to predict Cal prices using fb Prophet.
cal_df = state_time_series[state_time_series.RegionName.str.contains('California')]
cal_df_median_prices = cal_df[['Date','RegionName', 'MedianSoldPrice_AllHomes']].dropna()
cal_df_for_prophet = cal_df_median_prices[['Date','MedianSoldPrice_AllHomes']]
cal_df_for_prophet

In [ ]:
#Rename our columns per prophet's requirements
cal_df_for_prophet = cal_df_for_prophet.rename(columns={"Date":"ds", "MedianSoldPrice_AllHomes":"y"})
cal_df_for_prophet

In [ ]:
m = Prophet()
m.fit(cal_df_for_prophet)

In [ ]:
future = m.make_future_dataframe(periods=50, freq='M')
future.tail()

In [ ]:
forecast = m.predict(future)
forecast.tail()

In [ ]:
m.plot(forecast)

In [ ]:
m.plot_components(forecast)